In [1]:
import instaloader
from decouple import config
from IPython.display import clear_output
import pandas as pd
import time

C:\Users\Razer\anaconda3\lib\site-packages\win_unicode_console\__init__.py:31: RuntimeWarning: sys.stdin.encoding == 'cp1252', whereas sys.stdout.encoding == 'UTF-8', readline hook consumer may assume they are the same
  readline_hook.enable(use_pyreadline=use_pyreadline)


In [6]:
# change code accordingly & add insta credentials to .env file

account_num = 2
accounts = ['',2,3]
account = accounts[account_num]
USER = config(f'USER{account}',default='')
PASSWORD = config(f'PASSWORD{account}',default='')
print(USER)
L = instaloader.Instaloader()
L.login(USER, PASSWORD)

jamainelim


In [3]:
js_followers = pd.read_csv('followers.csv')['0']

# change the start & end index
start = 1000
end = 2000
path = 'js_followers_following'
file_name = f'{path}/edges_{start}_to_{end}'

# read exisiting file or create new one 
try:
    edges = pd.read_csv(f'{file_name}.csv')
    data = edges[['from','to']].to_dict('list')
    print("csv found")
    edges['from'].value_counts()
    x=edges.groupby('from')[['to']].count().reset_index().sort_values(by='to')
    print("accounts scraped (not private or rejected):", x[x['to']>1]['to'].count())
except:
    data = {'from':[], 'to':[]}
    print('csv not found, created new one')
    


csv found
accounts scraped (not private or rejected): 31


In [ ]:
exceptions = {}
account_pointer = 0

# stats
privates = 0
no_posts = 0
bad_ratio = 0

for idx, username in enumerate(js_followers):

    # sleep for awhile
    if idx % 50 == 0:
        time.sleep(300)
        accounts.append(account)
        account_num += 1
        account = accounts[account_num]
        USER = config(f'USER{account}',default='')
        PASSWORD = config(f'PASSWORD{account}',default='')
        L = instaloader.Instaloader()
        L.login(USER, PASSWORD)
        
    # change the number below
    if idx == end:break
    if idx < start:continue
    if username in data['from']:continue

    # Loader
    try:
        profile = instaloader.Profile.from_username(L.context, username)
        following = profile.get_followees()
    except Exception as e:
        if e in exceptions.keys():exceptions[e]+=1
        else:exceptions[e]=1

    # variables
    is_private = profile.is_private
    no_posts = profile.mediacount
    ratio = profile.followees/(profile.followers or not profile.followers)
    print(f'doing {username} who is following {profile.followees} accounts')
    print(f"Private:{is_private} Posts:{no_posts} Ratio:{round(ratio,1)}")

    # checks
    if is_private:privates+=1
    if no_posts==0:no_posts+=1
    if ratio>10:bad_ratio+=1
    if no_posts==0 or ratio > 10:
        data['from'].append(username)
        data['to'].append("jam.stones")
        continue

    # collect data
    for follower in following:
        data['from'].append(username)
        data['to'].append(follower.username)

    # add JS edge
    data['from'].append(username)
    data['to'].append("jam.stones")

    # save file
    pd.DataFrame(data).to_csv(f"{file_name}.csv")

    print(f'finished js follower {idx}/{len(js_followers)}')